#### Scenario


 2. 기준 인풋 데이터를 토대로(평가) 관광지에 대한 평가 평균값을 계산.
3. Top 레이팅을 평가.
------------------------------------------
탐색
1. 이름/탐색 기준으로 key 값 지정
2. 평균값 내림차순으로 Key값에 의한 sort
3. 원하는 지역 탐색시 도메인 + 쿼리에 검색내용 추가 : 링크로 이동할 수 있게끔 도와줌
4. 

              

###### 1. Program begins

###### 2. Program data set

2-0 ALL IMPORT LIBRARY

In [85]:
import requests
!pip install haversine
import haversine
import datetime
from dateutil.parser import parse

2-1 importing classes

2-2 Cal_Distance class

In [143]:
class Cal_Distance:
    def getLatLng(self,p_start,p_dn_name):
        import requests
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_s_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_s_Lat = float(p_s_places[0]['y'])
        p_s_Lng = float(p_s_places[0]['x'])
        p_s_LatLng=(p_s_Lat,p_s_Lng)

        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_dn_name)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_d_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_d_Lat = float(p_d_places[0]['y'])
        p_d_Lng = float(p_d_places[0]['x'])
        p_d_LatLng=(p_d_Lat,p_d_Lng)

        #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
        #상대적 직선거리 (relative straight distance 변수에 지정)
        from haversine import haversine
        r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
        return r_s_d

    


# select_number = int(input('몇 번째 주소인지 입력하세요 : '))

# print(places[select_number])
    

2-3 Evaluation Class

In [147]:
#  기준값적용 (평가기준값)
#  - 기준값 지정 (총 5점)
#  - 평가값을 받는 것 = 이동수단 / 비용 / 거리 ** 서울역 기준 /   

class Evaluation:
    def evalue_costs(self,p_cost,p_length_of_stay): #관광데이터랩 1회 평균 국내여행지출 금액 133,000원 정규분포도 활용
        if p_cost/p_length_of_stay <= 50000: #50000
            return 5
        elif 50000 < p_cost/p_length_of_stay <= 100000: #50001~100000
            return 4
        elif 100000 < p_cost/p_length_of_stay <= 150000: #100001~150000
            return 3
        elif 150000 < p_cost/p_length_of_stay <= 200000: #150001~200000
            return 2
        else: #200001이상.
            return 1
        
    def evalue_by_distance(self,p_transportation,r_s_d): #이동수단을 직선거리로 평가.
        if p_transportation == "도보":
            if r_s_d < 1:
                return 5
            elif r_s_d < 2:
                return 4
            elif r_s_d < 3:
                return 3
            elif r_s_d < 4:
                return 2
            else:
                return 1
            
        elif p_transportation == "지역버스":
            if r_s_d < 5:
                return 5
            elif r_s_d < 15:
                return 4
            elif r_s_d < 25:
                return 3
            elif r_s_d < 35:
                return 2
            else:
                return 1
            
        elif p_transportation == "자동차":
            if r_s_d < 10:
                return 5
            elif r_s_d < 30:
                return 4
            elif r_s_d < 50:
                return 3
            elif r_s_d < 70:
                return 2
            else:
                return 1
            
        elif p_transportation == "고속버스":
            if r_s_d < 20:
                return 5
            elif r_s_d < 50:
                return 4
            elif r_s_d < 80:
                return 3
            elif r_s_d < 110:
                return 2
            else:
                return 1
            
        elif p_transportation == "전철":
            if r_s_d < 40:
                return 5
            elif r_s_d < 70:
                return 4
            elif r_s_d < 100:
                return 3
            elif r_s_d < 130:
                return 2
            else:
                return 1
        
    def evalue_by_satisfaction(self,p_satisfaction): #관광만족도 개인 평가 
        if p_satisfaction <= 1:
            return 1
        elif 1 < p_satisfaction <= 2: 
            return 2
        elif 2 < p_satisfaction <= 3: 
            return 3
        elif 3 < p_satisfaction <= 4: 
            return 4
        else: 
            return 5
        
    
            
        

2-4 Record Class

2-4-1 Method(Evaluation,Cal_Distance) class 상속

In [151]:
class Method(Evaluation,Cal_Distance,Find):
    pass

In [153]:
m = Method()
m.date()

TypeError: date() missing 1 required positional argument: 'p_date'

2-4-2 Record(Method,Find) class 상속

In [169]:
class Record:
    def first_record(self):
        with open("Dummy.txt","w") as data:    
            print("첫번째 기록입니다.")
            while True:
                #Method 클래스 지정
                m = Method()
                #입력 데이터값 
                p_date = input("여행날짜 ex)20210701 : ")
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/이동")
                p_dn_name = input("여행목적지(관광지이름) : ")
                p_start = input("출발 장소를 입력하세요 : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철")
                p_length_of_stay = int(input("총 여행 기간 : __ 일"))
                p_cost = int(input("총 지출 금액 : __ 원"))
                p_satisfaction = int(input("관광만족도는 어떠셧습니까? <매우좋았음:5 / 좋았음:4 / 보통:3 / 별로:2 / 비추천:1> : "))
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                #데이터 기록 값
                past_data = [{'여행날짜':p_date},
                             {'관광지이름':p_dn_name},
                             {'관광유형':p_type},
                             {'출발지':p_start},
                             {'이동수단':p_transportation},
                             {'관광기간':p_length_of_stay},
                             {'총비용':p_cost},
                             {'만족도':p_satisfaction},
                             {'지출비용평가점수':ev_c},
                             {'이동비율평가점수':ev_d},
                             {'만족도평가점수':ev_s}]
                
                past_data = str(past_data)
                data.write(past_data +',''\n')
                Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                else:
                    print("기록이 종료됩니다.")
                    print("기록된 내용입니다. ")
                    break
        with open("Dummy.txt",'r') as data_r: data_r = data_r.read()
        return print(data_r,sep='\n')
                    
                        
                    
    def another_record(self):
        with open("Dummy.txt","a") as data:    
            print("기존 데이터에 기록합니다.")
            while True:
                #Method 클래스 지정
                m = Method()
                #입력 데이터값 
                p_date = input("여행날짜 ex)20210701 : ")
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/이동")
                p_dn_name = input("여행목적지(관광지이름) : ")
                p_start = input("출발 장소를 입력하세요 : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철")
                p_length_of_stay = int(input("총 여행 기간 : __ 일"))
                p_cost = int(input("총 지출 금액 : __ 원"))
                p_satisfaction = int(input("관광만족도는 어떠셧습니까? <매우좋았음:5 / 좋았음:4 / 보통:3 / 별로:2 / 비추천:1> : "))
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                #데이터 기록 값
                past_data = [{'여행날짜':p_date},
                             {'관광지이름':p_dn_name},
                             {'관광유형':p_type},
                             {'출발지':p_start},
                             {'이동수단':p_transportation},
                             {'관광기간':p_length_of_stay},
                             {'총비용':p_cost},
                             {'만족도':p_satisfaction},
                             {'지출비용평가점수':ev_c},
                             {'이동비율평가점수':ev_d},
                             {'만족도평가점수':ev_s}]
                
                past_data = str(past_data)
                data.write(past_data +',''\n')
                Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                else:
                    print("기록이 종료되었습니다.")
                    print("추가 기록된 내용입니다.")
                    break
        return print(past_data)
                    

In [170]:
class R_DATA(Method,Record):
    pass

r = R_DATA()
r.evalue_costs()

TypeError: evalue_costs() missing 2 required positional arguments: 'p_cost' and 'p_length_of_stay'

###### 3. Program start class

# Dummy

In [175]:

    def start_program():
        Question = input("처음 기록합니까?  예/아니오 : ")
        if Question == "예":
            r.first_record()
        else:
            Another_Question = input("기존 데이터에 기록하겠습니까? 예/아니오 : ")
            if Another_Question == "예":
                r.another_record()
            else:
                print('기록 프로그램을 종료합니다')
                print('사용해주셔서 감사합니다')

In [176]:
p = R_DATA()
p.start_program()

TypeError: start_program() takes 0 positional arguments but 1 was given

##### 2-1-1 날짜변환 class

class Find:
    def date(self,p_date):
        from dateutil.parser import parse
        p_date= parse(p_date)
        p_date = p_date.strftime('%Y-%m-%d-%a')
        return p_date

2-2. Data Calucation sp)L_S_D

###### 주소 가져오기 
간단한 명칭만 넣으면 주소를 가져옴

class Cal_Distance:
    def getLatLng(self,p_start,p_dn_name):
        import requests
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_s_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_s_Lat = float(p_s_places[0]['y'])
        p_s_Lng = float(p_s_places[0]['x'])
        p_s_LatLng=(p_s_Lat,p_s_Lng)

        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_dn_name)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_d_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_d_Lat = float(p_d_places[0]['y'])
        p_d_Lng = float(p_d_places[0]['x'])
        p_d_LatLng=(p_d_Lat,p_d_Lng)

        #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
        #상대적 직선거리 (relative straight distance 변수에 지정)
        from haversine import haversine
        r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
        return r_s_d

    


# select_number = int(input('몇 번째 주소인지 입력하세요 : '))

# print(places[select_number])
    

2-2. Data Standard "Evauation"

#  기준값적용 (평가기준값)
#  - 기준값 지정 (총 5점)
#  - 평가값을 받는 것 = 이동수단 / 비용 / 거리 ** 서울역 기준 /   

class Evaluation:
    def evalue_costs(self,p_cost,p_length_of_stay): #관광데이터랩 1회 평균 국내여행지출 금액 133,000원 정규분포도 활용
        if p_cost/p_length_of_stay <= 50000: #50000ㄴ
            return 5
        elif 50000 < p_cost/p_length_of_stay <= 100000: #50001~100000
            return 4
        elif 100000 < p_cost/p_length_of_stay <= 150000: #100001~150000
            return 3
        elif 150000 < p_cost/p_length_of_stay <= 200000: #150001~200000
            return 2
        else: #200001이상.
            return 1
        
    def evalue_by_distance(self,p_transportation,r_s_d): #이동수단을 직선거리로 평가.
        if p_transportation == "도보":
            if r_s_d < 1:
                return 5
            elif r_s_d < 2:
                return 4
            elif r_s_d < 3:
                return 3
            elif r_s_d < 4:
                return 2
            else:
                return 1
            
        elif p_transportation == "지역버스":
            if r_s_d < 5:
                return 5
            elif r_s_d < 15:
                return 4
            elif r_s_d < 25:
                return 3
            elif r_s_d < 35:
                return 2
            else:
                return 1
            
        elif p_transportation == "자동차":
            if r_s_d < 10:
                return 5
            elif r_s_d < 30:
                return 4
            elif r_s_d < 50:
                return 3
            elif r_s_d < 70:
                return 2
            else:
                return 1
            
        elif p_transportation == "고속버스":
            if r_s_d < 20:
                return 5
            elif r_s_d < 50:
                return 4
            elif r_s_d < 80:
                return 3
            elif r_s_d < 110:
                return 2
            else:
                return 1
            
        elif p_transportation == "전철":
            if r_s_d < 40:
                return 5
            elif r_s_d < 70:
                return 4
            elif r_s_d < 100:
                return 3
            elif r_s_d < 130:
                return 2
            else:
                return 1
        
    def evalue_by_satisfaction(self,p_satisfaction): #관광만족도 개인 평가 
        if p_satisfaction <= 1:
            return 1
        elif 1 < p_satisfaction <= 2: 
            return 2
        elif 2 < p_satisfaction <= 3: 
            return 3
        elif 3 < p_satisfaction <= 4: 
            return 4
        else: 
            return 5
        
    
            
        

#### Class 상속 (평가기준값이 r_s_d 클래스를 가져오고, 바꾸는거 없고 + PASS적용)

In [79]:
# class Method(Evaluation,Cal_Distance):
#     def __init__(self):
#         return


# Dummy

class Method(Evaluation,Cal_Distance,Find):
    def __init__(self):
        return
    
class Record:
    def first_record(self):
        with open("Dummy.txt","w") as data:    
            print("첫번째 기록입니다.")
            while True:
                # 클래스 지정
                m = Method()
                #입력 데이터값 
                #Date format 변경
                p_date = input("여행날짜 ex)20210701 : ")#datetime 정규식으로 나눌것
                p_date = m.date(p_date)
                print(p_date)
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/이동")
                p_dn_name = input("여행목적지(관광지이름) : ")
                p_start = input("출발 장소를 입력하세요 : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철")
                p_length_of_stay = int(input("총 여행 기간 : __ 일"))
                p_cost = int(input("총 지출 금액 : __ 원"))
                p_satisfaction = int(input("관광만족도는 어떠셧습니까? <매우좋았음:5 / 좋았음:4 / 보통:3 / 별로:2 / 비추천:1> : "))
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                #데이터 기록 값
                past_data = {'여행날짜':p_date,
                             '관광지이름':p_dn_name,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction,
                             '지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                past_data = str(past_data)
                data.write(past_data +'\n')
                Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                else:
                    print("기록이 종료됩니다.")
                    print("기록된 내용입니다. ")
                    break
        with open("Dummy.txt",'r') as data_r: data_r = data_r.read()
        return print(data_r,sep='\n')
                    
                        
                    
    def another_record(self):
        with open("Dummy.txt","a") as data:    
            print("기존 데이터에 기록합니다.")
            while True:
                #Method 클래스 지정
                m = Method()
                #입력 데이터값 
                p_date = int(input("여행날짜 : "))#datetime 정규식으로 나눌것
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/이동")
                p_dn_name = input("여행목적지(관광지이름) : ")
                p_start = input("출발 장소를 입력하세요 : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철")
                p_length_of_stay = int(input("총 여행 기간 : __ 일"))
                p_cost = int(input("총 지출 금액 : __ 원"))
                p_satisfaction = int(input("관광만족도는 어떠셧습니까? <매우좋았음:5 / 좋았음:4 / 보통:3 / 별로:2 / 비추천:1> : "))
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                #데이터 기록 값
                past_data = {'여행날짜':p_date,
                             '관광지이름':p_dn_name,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction,
                             '지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                past_data = str(past_data)
                data.write(past_data +'\n')
                Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                else:
                    print("기록이 종료되었습니다.")
                    print("추가 기록된 내용입니다.")
                    break
        return print(past_data)
                    

###### 위도경도 차이 구하기

!pip install haversine
from haversine import haversine
k = (37.504030,127.024099)
g = (37.497175,127.027926)
print(type(k[0]))
print(int(haversine(k,g)*1000),'km')

def getLatLng():
    import requests
    p_start = input("출발 장소(도로명주소) 를 입력하세요 : ")
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
    headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
    p_s_places = requests.get(url, headers = headers).json()['documents']
    p_s_Lat = float(p_s_places[0]['y'])
    p_s_Lng = float(p_s_places[0]['x'])
    p_s_LatLng=(p_s_Lat,p_s_Lng)
    
    p_destination = input("중요 관광지(도로명주소)를 입력하세요 : ")
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_destination)
    headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
    p_d_places = requests.get(url, headers = headers).json()['documents']
    p_d_Lat = float(p_d_places[0]['y'])
    p_d_Lng = float(p_d_places[0]['x'])
    p_d_LatLng=(p_d_Lat,p_d_Lng)
    
    #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
    #상대적 직선거리 (relative straight distance 변수에 지정)
    from haversine import haversine
    r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
    print(type(r_s_d))
    return r_s_d
    
    


# select_number = int(input('몇 번째 주소인지 입력하세요 : '))

# print(places[select_number])
    

In [83]:
class Program:
    def start_program(self):
        Question = input("처음 기록합니까?  예/아니오 : ")
        if Question == "예":
            a = Record()
            a.first_record()
        else:
            Another_Question = input("기존 데이터에 기록하겠습니까? 예/아니오 : ")
            if Another_Question == "예":
                a.another_record()
            else:
                print('기록 프로그램을 종료합니다')
                print('사용해주셔서 감사합니다')

In [84]:
p=Program()
p.start_program()

처음 기록합니까?  예/아니오 : 예
첫번째 기록입니다.
여행날짜 ex)20210701 : 20210701
2021-07-01-Thu
관광 유형을 고르세요. 도시/바다/캠핑/이동이동
여행목적지(관광지이름) : 서울역
출발 장소를 입력하세요 : 이대역
이동수단을 고르세요. 도보/지역버스/자차/고속버스/전철전철
총 여행 기간 : __ 일1
총 지출 금액 : __ 원1000
관광만족도는 어떠셧습니까? <매우좋았음:5 / 좋았음:4 / 보통:3 / 별로:2 / 비추천:1> : 3
다른 여행지를 기록하시겠습니까? 예/아니오 : 아니오
기록이 종료됩니다.
기록된 내용입니다. 
{'여행날짜': '2021-07-01-Thu', '관광지이름': '서울역', '관광유형': '이동', '출발지': '이대역', '이동수단': '전철', '관광기간': 1, '총비용': 1000, '만족도': 3, '지출비용평가점수': 5, '이동비율평가점수': 5, '만족도평가점수': 3}



In [ ]:
# class Distance:
#     def getLatLng(self):
#         #모듈 다운로드
# #         !pip install haversine
#         #출발장소 도로명주소 위/경도로 변환 --float값으로 변환
#         addr_1 = input("출발 장소(도로명주소) 를 입력하세요 : ")
#         url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr_1
#         headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
#         result = json.loads(str(requests.get(url, headers=headers).text))
# #         match_first = result['documents'][0]['address']
# #         first_destination_Lat = float(match_first['y'])
# #         first_destination_Lng = float(match_first['x'])
# #         first_LatLng = (first_destination_Lat,first_destination_Lng)
#         print(result)

# #         addr_2 = input("여행 장소(도로명주소) 를 입력하세요 : ")
# #         url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr_2
# #         headers = {"Authorization" : "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
# #         result = json.loads(str(requests.get(url, headers=headers).text))
# #         match_second = result['documents'][0]['address']
# #         second_destination_Lat = float(match_second['y'])
# #         second_destination_Lng = float(match_second['x'])
# #         second_LatLng = (second_destination_Lat,second_destination_Lng)

# #         #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
# #         #상대적 직선거리 (relative straight distance 변수에 지정)
# #         from haversine import haversine
# #         r_s_d = int(haversine(first_LatLng,second_LatLng))
# #         print(type(r_s_d))
# #         return r_s_d


In [ ]:
# from urllib.parse import quote_plus, urlencode
# from urllib.request import urlopen, Request
# import xml.etree.ElementTree as ET
# import json


# print('도로명주소 검색API 서비스를 이용한 주소검색결과를 보여줍니다.')
# keystr = input('검색어를 입력하세요. : ')
# resulttype = input('result type을 입력하세요(0 => xml, 1=> json) : ')

# if resulttype=='1':
#     resulttype= 'json'
# else:
#     resulttype = 'xml'

# #print(resulttype)

# url = 'http://www.juso.go.kr/addrlink/addrLinkApi.do'
# queryParams = '?' + urlencode({ quote_plus('currentPage') : '1' , 
#                                quote_plus('countPerPage') : '10', 
#                                quote_plus('resultType') : resulttype, 
#                                quote_plus('keyword') : keystr, 
#                                quote_plus('confmKey') : 'devU01TX0FVVEgyMDIxMDcwMTE3Mzg0NDExMTM0Nzk='})

# request = Request(url + queryParams)
# request.get_method = lambda: 'GET'
# response_body = urlopen(request).read()

# #print(response_body.decode('utf-8'))

# print("url = " + url)
# print("keyword = " + keystr)
# print("="*100)


# if resulttype=='json':
#     root_json = json.loads(response_body)
#     print('<< results >>')
#     print('totalCount   : ' + root_json['results']['common']['totalCount'])
#     print('currentPage  : ' + root_json['results']['common']['currentPage'])
#     print('countPerPage : ' + root_json['results']['common']['countPerPage'])
#     print('errorCode    : ' + root_json['results']['common']['errorCode'])
#     print('errorMessage : ' + root_json['results']['common']['errorMessage'])
#     for child in root_json['results']['juso']:
#         print('-'*100)
#         print('[' + child['zipNo'] + '] ' + child['roadAddr'])
#         print('    지번주소     = ' + child['jibunAddr'])
#         print('    영문주소     = ' + child['engAddr'])
#         print('    도로명코드   = ' + child['rnMgtSn'])
#         print('    건물관리번호 = ' + child['bdMgtSn'])
#         print('    법정동코드   = ' + child['admCd'])
#         print('    상세건물명   = ' + child['detBdNmList'])
#         print('')
    

# else:
#     root_xml = ET.fromstring(response_body)

#     print('<< ' + root_xml.tag + ' >>')

#     print('totalCount   : ' + root_xml.find('common').findtext('totalCount'))
#     print('currentPage  : ' + root_xml.find('common').findtext('currentPage'))
#     print('countPerPage : ' + root_xml.find('common').findtext('countPerPage'))
#     print('errorCode    : ' + root_xml.find('common').findtext('errorCode'))
#     print('errorMessage : ' + root_xml.find('common').findtext('errorMessage'))

#     for child in root_xml.findall('juso'):
#         print('-'*100)
#         print('[' + child.findtext('zipNo') + '] ' + child.findtext('roadAddr'))
#         print('    지번주소     = ' + child.findtext('jibunAddr'))
#         print('    영문주소     = ' + child.findtext('engAddr'))
#         print('    도로명코드   = ' + child.findtext('rnMgtSn'))
#         print('    건물관리번호 = ' + child.findtext('bdMgtSn'))
#         print('    법정동코드   = ' + child.findtext('admCd'))
#         print('    상세건물명   = ' + child.findtext('detBdNmList'))
#         print('')